In [1]:
 import spacy
from joblib import Parallel, delayed
import time


In [2]:
# Load SpaCy's English model
nlp = spacy.load('en_core_web_sm')

import spacy
from joblib import Parallel, delayed
import time
import pandas as pd

# Load SpaCy's English model
nlp = spacy.load('en_core_web_sm')



In [4]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/mbti_1.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
print(data.head())

   type                                              posts
0  INFJ  'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1  ENTP  'I'm finding the lack of me in these posts ver...
2  INTP  'Good one  _____   https://www.youtube.com/wat...
3  INTJ  'Dear INTP,   I enjoyed our conversation the o...
4  ENTJ  'You're fired.|||That's another silly misconce...


In [6]:
import pandas as pd
import re
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from tqdm import tqdm
import spacy

# Load SpaCy for lemmatization
nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Remove links, special characters
    text = re.sub(r'http\S+|www.\S+', '', text)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)         # Remove punctuation
    text = text.lower()                         # Lowercase text
    return text

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized = " ".join([token.lemma_ for token in doc if not token.is_stop])
    return lemmatized

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/mbti_1.csv')
start = time.time()

# Preprocess text (remove links, etc.)
tqdm.pandas()
data['preprocessed_text'] = data['posts'].progress_apply(preprocess_text)

# Lemmatize using progress_apply
data['processed_text'] = data['preprocessed_text'].progress_apply(lemmatize_text)

print("Preprocessing complete. Time (seconds):", time.time() - start)

100%|██████████| 8675/8675 [33:18<00:00,  4.34it/s]

Preprocessing complete. Time (seconds): 2002.2443234920502


In [7]:
# Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(data['processed_text']).toarray()
y = data['type']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Train the model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_balanced, y_train_balanced)

# Evaluate the model
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

        ENFJ       0.55      0.61      0.57        38
        ENFP       0.69      0.61      0.65       135
        ENTJ       0.43      0.50      0.46        46
        ENTP       0.58      0.62      0.60       137
        ESFJ       0.43      0.33      0.38         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.50      0.38      0.43         8
        ESTP       0.53      0.56      0.54        18
        INFJ       0.73      0.66      0.70       294
        INFP       0.71      0.76      0.73       366
        INTJ       0.64      0.62      0.63       218
        INTP       0.69      0.73      0.71       261
        ISFJ       0.55      0.52      0.53        33
        ISFP       0.58      0.52      0.55        54
        ISTJ       0.56      0.56      0.56        41
        ISTP       0.62      0.70      0.66        67

    accuracy                           0.66      1735
  

In [8]:
def predict_personality(input_text):
    processed_text = preprocess_text(input_text)  # Preprocess input
    vectorized_text = tfidf.transform([processed_text]).toarray()
    prediction = model.predict(vectorized_text)
    return prediction[0]

# Test with custom input
test_inputs = [
    "I love deep intellectual conversations and enjoy thinking about abstract concepts.",
    "I'm highly organized and like planning my tasks ahead.",
    "I enjoy socializing and trying new activities with friends."
]

for text in test_inputs:
    print(f"Input: {text}")
    print(f"Predicted Personality: {predict_personality(text)}\n")


Input: I love deep intellectual conversations and enjoy thinking about abstract concepts.
Predicted Personality: INTP

Input: I'm highly organized and like planning my tasks ahead.
Predicted Personality: INTJ

Input: I enjoy socializing and trying new activities with friends.
Predicted Personality: ISFP



In [9]:
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.7 MB/s eta 0:00:00


In [10]:
!pip install pyngrok

In [11]:
from pyngrok import ngrok

# Replace the string below with your actual authtoken
authtoken = "2nAUp5YnDqOJPFrgKfyCjLiJOiO_6Dmdt6jWfHTruszLzii7D"
ngrok.set_auth_token(authtoken)

In [12]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [13]:
# Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(data['processed_text']).toarray()
y = data['type']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Balance the dataset using SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Train the model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_balanced, y_train_balanced)

# Evaluate the model
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
import joblib

# ... (Your model training code) ...

# Save the TF-IDF vectorizer
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')

# Save the trained model
joblib.dump(model, 'personality_prediction_model.pkl')

Classification Report:
               precision    recall  f1-score   support

        ENFJ       0.55      0.61      0.57        38
        ENFP       0.69      0.61      0.65       135
        ENTJ       0.43      0.50      0.46        46
        ENTP       0.58      0.62      0.60       137
        ESFJ       0.43      0.33      0.38         9
        ESFP       0.00      0.00      0.00        10
        ESTJ       0.50      0.38      0.43         8
        ESTP       0.53      0.56      0.54        18
        INFJ       0.73      0.66      0.70       294
        INFP       0.71      0.76      0.73       366
        INTJ       0.64      0.62      0.63       218
        INTP       0.69      0.73      0.71       261
        ISFJ       0.55      0.52      0.53        33
        ISFP       0.58      0.52      0.55        54
        ISTJ       0.56      0.56      0.56        41
        ISTP       0.62      0.70      0.66        67

    accuracy                           0.66      1735
  

['personality_prediction_model.pkl']

In [14]:
%%writefile app.py
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import joblib
import numpy as np

# Load the pre-trained model and tokenizer for personality prediction
# Load your TF-IDF vectorizer and trained personality prediction model (Assuming they are saved as .pkl files)
tfidf = joblib.load('tfidf_vectorizer.pkl')
model = joblib.load('personality_prediction_model.pkl')

# Preprocessing function (Assume it's defined elsewhere)
def preprocess_text(text):
    # Implement text preprocessing logic here (lowercasing, removing stopwords, etc.)
    return text.lower()

# Function to predict personality
def predict_personality(input_text):
    processed_text = preprocess_text(input_text)  # Preprocess input
    vectorized_text = tfidf.transform([processed_text]).toarray()
    prediction = model.predict(vectorized_text)
    return prediction[0]

# Streamlit UI
st.title("Personality Predictor")
user_input = st.text_area("Enter text to predict personality:")

if st.button("Predict Personality"):
    if user_input.strip():  # Ensure the input is not empty
        prediction = predict_personality(user_input)
        st.write(f"Predicted Personality: {prediction}")
    else:
        st.write("Please enter some text to predict personality.")


Writing app.py


In [15]:
from pyngrok import ngrok

# List all active tunnels
tunnels = ngrok.get_tunnels()

# Disconnect each tunnel
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)
    print(f"Disconnected tunnel: {tunnel.public_url}")

In [ ]:
import subprocess
from pyngrok import ngrok

# Start the ngrok tunnel
# Specify tunnel type and port explicitly
tunnel = ngrok.connect(addr="8501", bind_tls=True)
print("Streamlit app URL:", tunnel.public_url)

# Run the Streamlit app
subprocess.run(["streamlit", "run", "app.py"])

Streamlit app URL: https://a91dbba995f3.ngrok-free.app
